# Thực hành phân loại văn bản sử dụng Machine learning
[![](https://nguyenvanhieu.vn/wp-content/uploads/2020/08/phan-loai-van-ban-tieng-viet.jpg)](https://nguyenvanhieu.vn/phan-loai-van-ban-tieng-viet/)

> Bài hướng dẫn: https://nguyenvanhieu.vn/phan-loai-van-ban-tieng-viet/



# Tiền xử lý dữ liệu
- Xóa HTML code (nếu có)
- Chuẩn hóa bảng mã Unicode
- Chuẩn hóa kiểu gõ dấu tiếng Việt
=> Xem thêm: [Xử lý tiếng Việt trong Python](https://nguyenvanhieu.vn/xu-ly-tieng-viet-trong-python/)
- Thực hiện tách từ tiếng Việt (sử dụng thư viện tách từ như pyvi, underthesea, vncorenlp,...)
- đưa về văn bản lower (viết thường)
- Xóa các ký tự đặc biệt: ".", ",", ";", ")", ...

## Một số tập dữ liệu có thể tham khảo
- https://github.com/duyvuleo/VNTC
- http://github.com/binhvq/news-corpus

> Phần này mình chỉ hướng dẫn cách xử lý 1 bài báo. Sau đó, mình sẽ cung cấp tập dữ liệu tin tức đã được xử lý

In [1]:
# Cài đặt một số hàm tiền xử lý văn bản cần thiết
# !pip3 install --user underthesea
import regex as re
from underthesea import word_tokenize
 
uniChars = "àáảãạâầấẩẫậăằắẳẵặèéẻẽẹêềếểễệđìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵÀÁẢÃẠÂẦẤẨẪẬĂẰẮẲẴẶÈÉẺẼẸÊỀẾỂỄỆĐÌÍỈĨỊÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢÙÚỦŨỤƯỪỨỬỮỰỲÝỶỸỴÂĂĐÔƠƯ"
unsignChars = "aaaaaaaaaaaaaaaaaeeeeeeeeeeediiiiiooooooooooooooooouuuuuuuuuuuyyyyyAAAAAAAAAAAAAAAAAEEEEEEEEEEEDIIIOOOOOOOOOOOOOOOOOOOUUUUUUUUUUUYYYYYAADOOU"
 
def loaddicchar():
    dic = {}
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split(
        '|')
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split(
        '|')
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic
dicchar = loaddicchar()

# Hàm chuyển Unicode dựng sẵn về Unicde tổ hợp (phổ biến hơn)
def convert_unicode(txt):
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)

bang_nguyen_am = [['a', 'à', 'á', 'ả', 'ã', 'ạ', 'a'],
                  ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ', 'aw'],
                  ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ', 'aa'],
                  ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ', 'e'],
                  ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ', 'ee'],
                  ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị', 'i'],
                  ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ', 'o'],
                  ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ', 'oo'],
                  ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ', 'ow'],
                  ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ', 'u'],
                  ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự', 'uw'],
                  ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ', 'y']]
bang_ky_tu_dau = ['', 'f', 's', 'r', 'x', 'j']

nguyen_am_to_ids = {}

for i in range(len(bang_nguyen_am)):
    for j in range(len(bang_nguyen_am[i]) - 1):
        nguyen_am_to_ids[bang_nguyen_am[i][j]] = (i, j)

def chuan_hoa_dau_tu_tieng_viet(word):
    if not is_valid_vietnam_word(word):
        return word

    chars = list(word)
    dau_cau = 0
    nguyen_am_index = []
    qu_or_gi = False
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x == -1:
            continue
        elif x == 9:  # check qu
            if index != 0 and chars[index - 1] == 'q':
                chars[index] = 'u'
                qu_or_gi = True
        elif x == 5:  # check gi
            if index != 0 and chars[index - 1] == 'g':
                chars[index] = 'i'
                qu_or_gi = True
        if y != 0:
            dau_cau = y
            chars[index] = bang_nguyen_am[x][0]
        if not qu_or_gi or index != 1:
            nguyen_am_index.append(index)
    if len(nguyen_am_index) < 2:
        if qu_or_gi:
            if len(chars) == 2:
                x, y = nguyen_am_to_ids.get(chars[1])
                chars[1] = bang_nguyen_am[x][dau_cau]
            else:
                x, y = nguyen_am_to_ids.get(chars[2], (-1, -1))
                if x != -1:
                    chars[2] = bang_nguyen_am[x][dau_cau]
                else:
                    chars[1] = bang_nguyen_am[5][dau_cau] if chars[1] == 'i' else bang_nguyen_am[9][dau_cau]
            return ''.join(chars)
        return word

    for index in nguyen_am_index:
        x, y = nguyen_am_to_ids[chars[index]]
        if x == 4 or x == 8:  # ê, ơ
            chars[index] = bang_nguyen_am[x][dau_cau]
            # for index2 in nguyen_am_index:
            #     if index2 != index:
            #         x, y = nguyen_am_to_ids[chars[index]]
            #         chars[index2] = bang_nguyen_am[x][0]
            return ''.join(chars)

    if len(nguyen_am_index) == 2:
        if nguyen_am_index[-1] == len(chars) - 1:
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            chars[nguyen_am_index[0]] = bang_nguyen_am[x][dau_cau]
            # x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            # chars[nguyen_am_index[1]] = bang_nguyen_am[x][0]
        else:
            # x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            # chars[nguyen_am_index[0]] = bang_nguyen_am[x][0]
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
    else:
        # x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
        # chars[nguyen_am_index[0]] = bang_nguyen_am[x][0]
        x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
        chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
        # x, y = nguyen_am_to_ids[chars[nguyen_am_index[2]]]
        # chars[nguyen_am_index[2]] = bang_nguyen_am[x][0]
    return ''.join(chars)


def is_valid_vietnam_word(word):
    chars = list(word)
    nguyen_am_index = -1
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x != -1:
            if nguyen_am_index == -1:
                nguyen_am_index = index
            else:
                if index - nguyen_am_index != 1:
                    return False
                nguyen_am_index = index
    return True


def chuan_hoa_dau_cau_tieng_viet(sentence):
    """
        Chuyển câu tiếng việt về chuẩn gõ dấu kiểu cũ.
        :param sentence:
        :return:
        """
    sentence = sentence.lower()
    words = sentence.split()
    for index, word in enumerate(words):
        cw = re.sub(r'(^\p{P}*)([p{L}.]*\p{L}+)(\p{P}*$)', r'\1/\2/\3', word).split('/')
        # print(cw)
        if len(cw) == 3:
            cw[1] = chuan_hoa_dau_tu_tieng_viet(cw[1])
        words[index] = ''.join(cw)
    return ' '.join(words)

def remove_html(txt):
    return re.sub(r'<[^>]*>', '', txt)

In [ ]:
def text_preprocess(document):
    # xóa html code
    document = remove_html(document)
    # chuẩn hóa unicode
    document = convert_unicode(document)
    # chuẩn hóa cách gõ dấu tiếng Việt
    document = chuan_hoa_dau_cau_tieng_viet(document)
    # tách từ
    document = word_tokenize(document, format="text")
    # đưa về lower
    document = document.lower()
    # xóa các ký tự không cần thiết
    document = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_]',' ',document)
    # xóa khoảng trắng thừa
    document = re.sub(r'\s+', ' ', document).strip()
    return document

document = """
TP HCM phạt người không đeo khẩu trang nơi công cộng
Người dân ở thành phố không đeo khẩu trang nơi công cộng sẽ bị xử phạt mức cao nhất 300.000 đồng, từ ngày 5/8.

Yêu cầu này được Chủ tịch UBND thành phố Nguyễn Thành Phong đưa ra tại cuộc họp Ban chỉ đạo phòng chống dịch bệnh Covid-19 của TP HCM chiều 3/8.

Việc xử phạt không đeo khẩu trang nơi công cộng được TP HCM cũng như các địa phương khác thực hiện từ cuối tháng 3 khi Covid-19 bùng phát. Tuy nhiên, sau khi hết thực hiện cách ly xã hội từ ngày 23/4, việc đeo khẩu trang nơi công cộng chỉ dừng lại ở mức khuyến cáo.

Theo Nghị định số 176/2013, người dân không đeo khẩu trang nơi công cộng sẽ bị xử phạt từ 100.000 đến 300.000 đồng. Trong khoảng một tháng áp dụng trước đó, TP HCM đã xử phạt hơn 4.300 trường hợp với gần 870 triệu đồng.

Theo ông Phong, việc đeo khẩu trang đã được khẳng định có thể tránh lây lan dịch bệnh cho người khác và bảo vệ sức khỏe cho người sử dụng. "Sở Công thương phải nắm nguồn cung ứng khẩu trang, chủ động thông báo các điểm bán để người dân dễ dàng mua vì đã xử phạt thì phải bảo đảm đủ nguồn cung", ông Phong nói.

Đội trật tự đô thị phường Bến Nghé, quận 1, xử phạt người không đeo khẩu trang trên phố đi bộ Nguyễn Huệ, chiều 15/4. Ảnh: Quỳnh Trần.
Đội trật tự đô thị phường Bến Nghé, quận 1, xử phạt người không đeo khẩu trang trên phố đi bộ Nguyễn Huệ, chiều 15/4. Ảnh: Quỳnh Trần.

Bí thư Thành uỷ Nguyễn Thiện Nhân cũng cho rằng việc đeo khẩu trang là một trong những biện pháp cơ bản để tránh dịch bệnh lây lan. Việc này rất dễ làm, không tốn nhiều tiền nhưng nhiều nước bỏ lơi và đã bị "vỡ trận".

"Ngoài đường hiện có ít nhất 20% người không đeo khẩu trang. Người không đeo không những tự rước bệnh vào mình mà còn nguy cơ lây cho người khác. Đeo khẩu trang hơi cực tí thôi nhưng đi đâu cũng nên đeo để giữ an toàn", ông Nhân nói và khẳng định thành phố bảo đảm không thiếu khẩu trang cho người dân.

Chủ tịch UBND thành phố Nguyễn Thành Phong cũng cho biết đã đồng ý việc tái lập các chốt kiểm soát ở cửa ngõ TP HCM để phòng chống Covid-19.

Trước đó, thành phố đã lập 62 chốt kiểm soát, hoạt động 24/24 từ ngày 4/4 để phòng chống dịch. Lực lượng tham gia là Công an thành phố, Sở Y tế, Bộ Tư lệnh thành phố, Thanh tra giao thông, Ban Quản lý An toàn thực phẩm, quản lý thị trường.

Trong đó, 16 chốt chính (cấp thành phố) đặt tại: Trạm thu phí Long Phước (cao tốc TP HCM - Long Thành - Dầu Giây), cao tốc Trung Lương, cầu Đôi (đường Trần Văn Giàu), đường Ba Làng, đường Xuyên Á (quốc lộ 22), cầu Phú Cường, cầu Vĩnh Bình, cầu vượt Sóng Thần, quốc lộ 1K, quốc lộ 50, quốc lộ 1A, cầu Đồng Nai, Bến xe Miền Tây, Bến xe miền Đông, sân bay Tân Sơn Nhất, cảng Cát Lái.

Đến ngày 23/4, chính quyền thành phố dừng hoạt động các chốt này vì dịch bệnh đã được khống chế, TP HCM dừng cách ly xã hội theo Chỉ thị 19 của Thủ tướng.

Sau 19 ngày hoạt động, các chốt chính đã kiểm tra gần 270.000 xe, trong đó có 235.000 ôtô; gần 600.000 người được kiểm tra y tế, đo thân nhiệt, bao gồm cả 3.000 người nước ngoài; hơn 130.000 người được yêu cầu khai báo y tế.
"""

document = text_preprocess(document)
print(document)

tp hcm phạt người không đeo khẩu_trang nơi công_cộng người dân ở thành_phố không đeo khẩu_trang nơi công_cộng sẽ bị xử_phạt mức cao nhất 300 000 đồng từ ngày 58 yêu_cầu này được chủ_tịch ubnd thành_phố nguyễn_thành phong đưa ra tại cuộc họp ban chỉ_đạo phòng_chống dịch_bệnh covid 19 của tp hcm chiều 38 việc xử_phạt không đeo khẩu_trang nơi công_cộng được tp hcm cũng như các địa_phương khác thực_hiện từ cuối tháng 3 khi covid 19 bùng_phát tuy_nhiên sau khi hết thực_hiện cách_ly xã_hội từ ngày 234 việc đeo khẩu_trang nơi công_cộng chỉ dừng lại ở mức khuyến_cáo theo nghị_định số 1762013 người_dân không đeo khẩu_trang nơi công_cộng sẽ bị xử_phạt từ 100 000 đến 300 000 đồng trong khoảng một tháng áp_dụng trước đó tp hcm đã xử_phạt hơn 4 300 trường_hợp với gần 870 triệu đồng theo ông phong việc đeo khẩu_trang đã được khẳng_định có_thể tránh lây_lan dịch_bệnh cho người khác và bảo_vệ sức_khỏe cho người sử_dụng sở công_thương phải nắm nguồn cung_ứng_khẩu_trang chủ_động thông_báo các điểm bán đ

# Thực hành phân loại văn bản
- Tải tập dữ liệu đã tiền xử lý về
- Quan sát tập dữ liệu
- Loại bỏ stopword

- Xây dựng tập train/test

- Phân loại văn bản sử dụng thuật toán Naive Bayes
- Phân loại văn bản sử dụng thuật toán SVM
- Phân loại văn bản sử dụng Fasttext

- Tổng kết/ Nhận xét

In [1]:
# Download data
# Dữ liệu đã tiền xử lý có dạng: mỗi bài báo là một dòng, từ đầu tiên là nhãn (chủ đề) của dòng đó
!wget -nc "https://github.com/nguyenvanhieuvn/text-classification-tutorial/raw/master/news_categories.zip"
!unzip -n "news_categories.zip"
!head "news_categories.txt"

uyên gặp_gỡ tình_yêu của mình hot boy tuấn_kiệt sau những lần chụp ảnh cho anh cô đã đem_lòng yêu chàng hot boy với nụ_cười tỏa nắng này tuy_nhiên khoảng_cách giữa hai người quá lớn miu lê chỉ là một cô_gái làm nghề nhiếp_ảnh cá_tính còn tuấn_kiệt lại là một hot boy điển_trai với tương_lai rộng_mở điều quan_trọng nhất là trước khi biết cô nhiếp_ảnh chàng_trai đã tìm được một_nửa của mình chấp_nhận là người đến sau miu lê chôn giấu tình_yêu sâu trong lòng và cầu_chúc cho tình_yêu của người_thương mỗi lúc nhớ tuấn_kiệt miu_lê thường ngắm lại những bức hình đã chụp về anh tất_cả những tấm ảnh đó chỉ làm cô thêm đau_lòng và cào xé tâm_can cuối_cùng cô quyết_định đi xa để tìm sự bình_yên trong tâm_hồn will 365 trong nơi anh không thuộc về nơi anh không thuộc về là một trong những mv khá đặc_biệt của 365 vì đây là lần đầu_tiên ngô_thanh_vân cho_phép các học_trò tình_tứ với gái lạ với mô típ mới 5 chàng_trai của 365 hóa_thân thành những hình_tượng khác nhau đáng chú_ý nhất phải kể đến will vớ

In [4]:
# Thống kê số lượng data theo nhãn
count = {}
for line in open('news_categories.txt'):
    key = line.split()[0]
    count[key] = count.get(key, 0) + 1

for key in count:
    print(key, count[key])

__label__thể_thao 2611
__label__âm_nhạc 2595
__label__nhịp_sống 2613
__label__công_nghệ 2595
__label__thời_sự 2607
__label__thế_giới 2602
__label__thời_trang 2596
__label__du_lịch 2593
__label__sống_trẻ 2602
__label__giáo_dục 2603
__label__kinh_doanh 2597
__label__pháp_luật 2592
__label__giải_trí 2604
__label__phim_ảnh 2596
__label__xe_360 2602
__label__ẩm_thực 2482
__label__xuất_bản 2599
__label__sức_khỏe 2589


In [5]:
# Thống kê các word xuất hiện ở tất cả các nhãn
total_label = 18
vocab = {}
label_vocab = {}
for line in open('news_categories.txt'):
    words = line.split()
    # lưu ý từ đầu tiên là nhãn
    label = words[0]
    if label not in label_vocab:
        label_vocab[label] = {}
    for word in words[1:]:
        label_vocab[label][word] = label_vocab[label].get(word, 0) + 1
        if word not in vocab:
            vocab[word] = set()
        vocab[word].add(label)

count = {}
for word in vocab:
    if len(vocab[word]) == total_label:
        count[word] = min([label_vocab[x][word] for x in label_vocab])
        
sorted_count = sorted(count, key=count.get, reverse=True)
for word in sorted_count[:100]:
    print(word, count[word])

và 14255
của 13177
là 9983
có 9162
được 9131
trong 8654
một 7575
cho 7483
với 7195
không 6591
các 6300
người 6088
khi 6011
này 5301
đến 5165
để 5123
đã 4431
nhiều 4167
trên 3842
từ 3820
vào 3617
đó 3207
những 3097
ở 2943
ra 2767
tại 2756
vn 2738
lại 2673
cũng 2622
phải 2615
còn 2589
theo 2565
nhưng 2545
zing 2519
về 2170
sau 2047
làm 1983
lên 1921
hơn 1755
đây 1692
năm 1654
sẽ 1597
chỉ 1569
cả 1534
cùng 1490
ngày 1484
như 1467
mà 1458
vẫn 1386
đi 1373
2 1358
mới 1357
khác 1300
3 1296
hai 1278
qua 1250
bạn 1138
bên 1137
1 1136
khiến 1122
5 1114
lần 1051
mình 1045
lớn 1030
bị 1022
biết 1013
trước 1000
rất 991
tới 968
bằng 948
mang 929
nên 897
4 896
đang 870
nước 863
cách 863
việt_nam 861
đầu 849
10 847
việc 840
nếu 835
vừa 826
thấy 824
hàng 807
vì 806
ảnh 799
đều 796
nhau 788
thời_gian 787
sự 764
anh 760
6 737
nhất 733
ngoài 720
điều 712
hay 706
giữa 699
số 699
từng 697
thêm 692


In [6]:
# loại stopword khỏi dữ liệu
# lưu file dùng về sau
stopword = set()
with open('stopwords.txt', 'w') as fp:
    for word in sorted_count[:100]:
        stopword.add(word)
        fp.write(word + '\n')
    
def remove_stopwords(line):
    words = []
    for word in line.strip().split():
        if word not in stopword:
            words.append(word)
    return ' '.join(words)
    
    
with open('news_categories.prep', 'w') as fp:
    for line in open('news_categories.txt'):
        line = remove_stopwords(line)
        fp.write(line + '\n')

In [7]:
!head news_categories.prep

__label__thể_thao sanchez dự trận khai màn nh arsenal bóng_đá tuyển_thủ chile kịp hồi_phục thể_lực pháo thủ_thành london tham_dự trận đầu_tiên premier_league 2015 2016 diễn 9 8 arsenal singapore liverpool đổ_bộ xuống thái_lan đội_bóng danh_tiếng giải ngoại_hạng_anh bắt_đầu chuyến du_đấu châu á thầy_trò hlv brendan_rodgers có_mặt thái_lan chuẩn_bị trận_đấu diễn 14 7 trả_lời phỏng_vấn chuyến du_đấu singapore hlv arsene_wenger xác_nhận tiền_đạo alexis_sanchez vắng_mặt trận khai màn premier_league chiến_lược gia pháp cựu ngôi_sao barca cần nghỉ_ngơi tham_dự copa_america quê_hương alexis trở_lại arsenal 8 cầu_thủ thường lấy thể_lực khoảng tuần có_thể thi_đấu alexis có_thể nghỉ ít tuy_nhiên ấy chắc_chắn tham_dự trận_đấu đầu_tiên mùa_giải trang bbc dẫn bài phỏng_vấn hlv wenger trận xông đất premier_league sanchez góp_mặt trận tranh community_shield chelsea bởi trận_đấu diễn 8 chân_sút 26 tuổi trở_lại arsenal sanchez nhân_tố then_chốt giúp chile vô_địch copa_america bbc mùa_giải vừa_qua cựu ti

In [2]:
# Chia tập train/test
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
test_percent = 0.2

text = []
label = []

for line in open('news_categories.prep'):
    words = line.strip().split()
    label.append(words[0])
    text.append(' '.join(words[1:]))

X_train, X_test, y_train, y_test = train_test_split(text, label, test_size=test_percent, random_state=42)

# Lưu train/test data
# Giữ nguyên train/test để về sau so sánh các mô hình cho công bằng
with open('train.txt', 'w') as fp:
    for x, y in zip(X_train, y_train):
        fp.write('{} {}\n'.format(y, x))

with open('test.txt', 'w') as fp:
    for x, y in zip(X_test, y_test):
        fp.write('{} {}\n'.format(y, x))

# encode label
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
print(list(label_encoder.classes_), '\n')
y_train = label_encoder.transform(y_train)
y_test = label_encoder.transform(y_test)

print(X_train[0], y_train[0], '\n')
print(X_test[0], y_test[0])

FileNotFoundError: [Errno 2] No such file or directory: 'news_categories.prep'

In [1]:
label_encoder

NameError: name 'label_encoder' is not defined

In [9]:
MODEL_PATH = "models"

import os
if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)


In [12]:
import pickle
import time
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

## Naive Bayes
https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html

In [ ]:
import pickle
import time
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

start_time = time.time()
text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,1),
                                             max_df=0.8,
                                             max_features=None)), 
                     ('tfidf', TfidfTransformer()), 
                     ('clf', MultinomialNB())
                    ])
text_clf = text_clf.fit(X_train, y_train)

train_time = time.time() - start_time
print('Done training Naive Bayes in', train_time, 'seconds.')

# Save model
pickle.dump(text_clf, open(os.path.join(MODEL_PATH, "naive_bayes.pkl"), 'wb'))

Done training Naive Bayes in 15.001639127731323 seconds.


## Linear Classifier
http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

In [13]:
from sklearn.linear_model import LogisticRegression

start_time = time.time()
text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,1),
                                             max_df=0.8,
                                             max_features=None)), 
                     ('tfidf', TfidfTransformer()),
                     ('clf', LogisticRegression(solver='lbfgs', 
                                                multi_class='auto',
                                                max_iter=10000))
                    ])
text_clf = text_clf.fit(X_train, y_train)

train_time = time.time() - start_time
print('Done training Linear Classifier in', train_time, 'seconds.')

# Save model
pickle.dump(text_clf, open(os.path.join(MODEL_PATH, "linear_classifier.pkl"), 'wb'))

Done training Linear Classifier in 263.68895626068115 seconds.


## Support Vector Machine (SVM)
https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

In [ ]:
from sklearn.svm import SVC

start_time = time.time()
text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,1),
                                             max_df=0.8,
                                             max_features=None)), 
                     ('tfidf', TfidfTransformer()),
                     ('clf', SVC(gamma='scale'))
                    ])
text_clf = text_clf.fit(X_train, y_train)

train_time = time.time() - start_time
print('Done training SVM in', train_time, 'seconds.')

# Save model
pickle.dump(text_clf, open(os.path.join(MODEL_PATH, "svm.pkl"), 'wb'))

Done training SVM in 2082.0848298072815 seconds.


## FastText
https://github.com/facebookresearch/fasttext

In [ ]:
# Cài đặt fastText cho Python
!pip3 install --user numpy scipy pybind11 cmake
!wget -nc "https://github.com/facebookresearch/fastText/archive/v0.9.2.zip" > /dev/null
!unzip -n "v0.9.2.zip" > /dev/null
!cd fastText-0.9.2 && pip3 install --user .
!rm -rf v0.9.2.zip fastText-0.9.2

You are using pip version 19.0.3, however version 20.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
--2020-08-05 23:07:26--  https://github.com/facebookresearch/fastText/archive/v0.9.2.zip
Resolving github.com (github.com)... 52.74.223.119
Connecting to github.com (github.com)|52.74.223.119|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/facebookresearch/fastText/zip/v0.9.2 [following]
--2020-08-05 23:07:27--  https://codeload.github.com/facebookresearch/fastText/zip/v0.9.2
Resolving codeload.github.com (codeload.github.com)... 54.251.140.56
Connecting to codeload.github.com (codeload.github.com)|54.251.140.56|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘v0.9.2.zip’

v0.9.2.zip              [           <=>      ]   4.17M  1.97MB/s    in 2.1s    

2020-08-05 23:07:30 (1.97 MB/s) - ‘v0.9.2.zip’ saved [4369852]

Proce

In [ ]:
import fasttext

start_time = time.time()
model = fasttext.train_supervised(
                                input='train.txt',
                                dim=100,
                                epoch=5,
                                lr=0.1,
                                wordNgrams=2,
                                label='__label__',
                                minCount=5
)

train_time = time.time() - start_time
print('Done training Fasttext in', train_time, 'seconds.')

# Compress model files with quantization
model.quantize(input='train.txt', retrain=True)
model.save_model(os.path.join(MODEL_PATH,"fasttext.ftz"))

Done training Fasttext in 19.906031131744385 seconds.


## Đánh giá các mô hình

In [ ]:
import numpy as np
# Naive Bayes
model = pickle.load(open(os.path.join(MODEL_PATH,"naive_bayes.pkl"), 'rb'))
y_pred = model.predict(X_test)
print('Naive Bayes, Accuracy =', np.mean(y_pred == y_test))

# Linear Classifier
model = pickle.load(open(os.path.join(MODEL_PATH,"linear_classifier.pkl"), 'rb'))
y_pred = model.predict(X_test)
print('Linear Classifier, Accuracy =', np.mean(y_pred == y_test))

# SVM
model = pickle.load(open(os.path.join(MODEL_PATH,"svm.pkl"), 'rb'))
y_pred = model.predict(X_test)
print('SVM, Accuracy =', np.mean(y_pred == y_test))

# Fasttext
def print_results(N, p, r):
    print("Fasttext, Precision = {}, Recall = {}".format(p, r))
    
model = fasttext.load_model(os.path.join(MODEL_PATH,"fasttext.ftz"))
print_results(*model.test('test.txt'))

Naive Bayes, Accuracy = 0.8324764353041988
Linear Classifier, Accuracy = 0.883140531276778
SVM, Accuracy = 0.8822836332476436


Fasttext, Precision = 0.7548200514138818, Recall = 0.7548200514138818


In [14]:
# Xem kết quả trên từng nhãn
from sklearn.metrics import classification_report

nb_model = pickle.load(open(os.path.join(MODEL_PATH,"linear_classifier.pkl"), 'rb'))
y_pred = nb_model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=list(label_encoder.classes_)))

                     precision    recall  f1-score   support

 __label__công_nghệ       0.94      0.94      0.94       532
   __label__du_lịch       0.88      0.90      0.89       551
  __label__giáo_dục       0.86      0.88      0.87       528
  __label__giải_trí       0.76      0.77      0.76       487
__label__kinh_doanh       0.86      0.86      0.86       498
 __label__nhịp_sống       0.81      0.79      0.80       497
  __label__phim_ảnh       0.87      0.87      0.87       525
 __label__pháp_luật       0.94      0.92      0.93       543
  __label__sống_trẻ       0.77      0.72      0.74       510
  __label__sức_khỏe       0.86      0.90      0.88       496
  __label__thế_giới       0.90      0.91      0.90       549
  __label__thể_thao       0.96      0.97      0.96       508
   __label__thời_sự       0.85      0.85      0.85       496
__label__thời_trang       0.88      0.88      0.88       521
    __label__xe_360       0.95      0.97      0.96       502
  __label__xuất_bản    

In [1]:
# xem kết quả cho 1 văn bản model naive bayes đã load ở trên

document = "đá bóng không mày?"

document = text_preprocess(document)
document = remove_stopwords(document)

label = nb_model.predict([document])
print('Predict label:', label_encoder.inverse_transform(label))

NameError: name 'text_preprocess' is not defined